In [14]:
import numpy as np
import os
import json
import pandas as pd
print(pd.__version__)

2.2.0


In [15]:
import hana_ml
from hana_ml import ConnectionContext
print(hana_ml.__version__)

# import llm_commons
# print(llm_commons.__version__)

2.19.24022101


### STEP 2: Connect to HANA DB

In [16]:
# with open(os.path.join(os.getcwd(), '/Users/i551982/Desktop/Github/RAG-demo-kr-1/config/HANA-configure.json')) as f:
with open(os.path.join(os.getcwd(), '/Users/i551982/Desktop/Github/enablement-session-01/HANA-configure-chul.json')) as f:

    hana_env_c = json.load(f)
    port_c  = hana_env_c['port']
    user_c  = hana_env_c['user']
    url_c  = hana_env_c['url']
    pwd_c  = hana_env_c['pwd']

cc = ConnectionContext(address=url_c, port=port_c, user= user_c, password = pwd_c, encrypt=True)
print(cc.hana_version())

cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")
print(cc.get_current_schema())

4.00.000.00.1707826119 (CE2024.4)
GEN_AI


In [17]:
cc.get_tables()

,TABLE_NAME
0,DEMO_BLOCK_1
1,DEMO_BLOCK_2
2,DEMO_BLOCK_3
3,DEMO_BLOCK_TEST
4,TEST_DEMO_BLOCK
5,VECTOR_TEST


In [18]:
cc.sql('''SELECT TOP 10 *  FROM DEMO_BLOCK_3''').collect().head()

,BlockID,MaterialNumber,MaterialName,MaterialDescription,MaterialDescription_KR,PurchasingGroup,PurchasingGroupDescription,PurchasingGroupDescription_KR,Total_stock,Unrestricted_use_stock,Blocked_stock,Blocked_date,Block_reason,Block_reason_KR,Solution,Solution_KR,Solution_date,VECTOR_EN,VECTOR_KR
0,5692,19011777,MELLTORP,"Table, 125x75 cm","테이블, 125x75 cm",P14,Tables & desks,테이블 & 책상,40906,2377,38529,2024-05-12,Inventory discrepancy detected during stock au...,재고 감사 중에 재고 불일치가 감지되었습니다.,"[{""Instruction_step"": 1, ""Instruction"": ""Inves...","[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-08-22,"[0, 6, 0, 0, 121, 71, 174, 59, 194, 182, 182, ...","[0, 6, 0, 0, 113, 133, 28, 188, 196, 138, 14, ..."
1,5692,19011777,MELLTORP,"Table, 125x75 cm","테이블, 125x75 cm",P14,Tables & desks,테이블 & 책상,40906,2377,38529,2024-05-12,Inventory discrepancy detected during stock au...,재고 감사 중에 재고 불일치가 감지되었습니다.,"[{""Instruction_step"": 1, ""Instruction"": ""Inves...","[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-08-22,"[0, 6, 0, 0, 121, 71, 174, 59, 194, 182, 182, ...","[0, 6, 0, 0, 113, 133, 28, 188, 196, 138, 14, ..."
2,5696,60214159,MALM,"Desk, 140x65 cm","책상, 140x65 cm",P14,Tables & desks,테이블 & 책상,22059,8914,13145,2024-05-15,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-06-15,"[0, 6, 0, 0, 193, 28, 239, 187, 98, 34, 73, 18...","[0, 6, 0, 0, 241, 121, 37, 187, 179, 74, 92, 1..."
3,5696,60214159,MALM,"Desk, 140x65 cm","책상, 140x65 cm",P14,Tables & desks,테이블 & 책상,22059,8914,13145,2024-05-15,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-06-15,"[0, 6, 0, 0, 193, 28, 239, 187, 98, 34, 73, 18...","[0, 6, 0, 0, 241, 121, 37, 187, 179, 74, 92, 1..."
4,5697,60214159,MALM,"Desk, 140x65 cm","책상, 140x65 cm",P14,Tables & desks,테이블 & 책상,22059,8914,13145,2024-05-15,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 손상시키는 포장 결함을 식별하여 즉각적인 수정 조치가 필...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-06-15,"[0, 6, 0, 0, 193, 28, 239, 187, 98, 34, 73, 18...","[0, 6, 0, 0, 183, 204, 88, 187, 64, 185, 43, 1..."


In [19]:
#
#     sql = '''SELECT TOP {k} "IssueID", "IssueDescription","IssueDate","MaterialNumber", "MaterialType", "MaterialDescription","SolutionDescription", "PurchasingGroup",	"PurchasingGroupDescription"
#         FROM "DEMO_BLOCK_1"
#         ORDER BY "{metric}"("VECTOR_EN", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
#


### STEP 3: Get Embeddings in a function

In [20]:
# get embedding
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="dc872f9eef04c31a") -> str:
    response = embeddings.create(
      deployment_id=model,
      input=input
    )
    return response.data[0].embedding



In [21]:
get_embedding("NICE TO MEET YOU")

[-0.011954373680055141,
 -0.0048379236832261086,
 0.011847075074911118,
 -0.03638068214058876,
 -0.0036923489533364773,
 0.04190973564982414,
 -0.0020197464618831873,
 0.014163471758365631,
 -0.008262025192379951,
 -0.021838508546352386,
 0.0005124317249283195,
 -0.00041144443093799055,
 -0.0011574093950912356,
 0.0012804876314476132,
 0.016359945759177208,
 -0.007637165952473879,
 0.024994362145662308,
 -0.0014296016888692975,
 0.014150847680866718,
 0.0036797253414988518,
 0.0042256880551576614,
 0.011493619531393051,
 -0.0006714078481309116,
 -0.009789458476006985,
 -0.011253774166107178,
 -0.01983138546347618,
 0.007106982637196779,
 -0.027443304657936096,
 -0.0019992333836853504,
 -0.010224966332316399,
 0.003166899085044861,
 -0.0008512915228493512,
 -0.0026524949353188276,
 0.004601234570145607,
 -0.01588025502860546,
 -0.03380550444126129,
 -0.01661241427063942,
 0.0011424190597608685,
 -0.0024773450568318367,
 -0.012251024134457111,
 0.024943867698311806,
 -0.01363328844308853

### STEP 4: Create HANA Vector Search in a function

In [22]:
# Wrapping HANA vector search in a function
def run_vector_search_en(query: str, metric="COSINE_SIMILARITY", k=5):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
        col = 'L2D_SIM'
    else:
        sort = 'DESC'
        col = 'COS_SIM'
    query_vector = get_embedding(query)

    sql = '''SELECT TOP {k} "BlockID","MaterialNumber", "MaterialName","MaterialDescription","PurchasingGroupDescription",
            "Blocked_stock","Block_reason","Blocked_date", "Solution", "Solution_date",
            "{metric}"("VECTOR_EN", TO_REAL_VECTOR('{qv}'))
            AS "{col}"
            FROM "DEMO_BLOCK_3"
            ORDER BY "{col}"'''.format(k=k, metric = metric, qv=query_vector, sort=sort, col = col)
            
    hdf = cc.sql(sql)
    df_context = hdf.collect()#hdf.head(k).collect()
    return df_context

In [23]:
# Wrapping HANA vector search in a function
def run_vector_search_kr(query: str, metric="COSINE_SIMILARITY", k=5):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
        col = 'L2D_SIM'
    else:
        sort = 'DESC'
        col = 'COS_SIM'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "BlockID","MaterialNumber", "MaterialName","MaterialDescription_KR","PurchasingGroupDescription_KR",
            "Blocked_stock","Block_reason_KR","Blocked_date", "Solution_KR", "Solution_date",
            "{metric}"("VECTOR_EN", TO_REAL_VECTOR('{qv}'))
            AS "{col}"
            FROM "DEMO_BLOCK_3"
            ORDER BY "{col}"'''.format(k=k, metric=metric, qv=query_vector, sort=sort, col = col)
    hdf = cc.sql(sql)
    df_context = hdf.collect() #hdf.head(k).collect()
    return df_context

In [24]:
# Test the vector search
query =  "What is the quality issue for packing defects for cabinet?"
df_context_en = run_vector_search_en(query=query, metric="COSINE_SIMILARITY",k=5)
df_context_en

,BlockID,MaterialNumber,MaterialName,MaterialDescription,PurchasingGroupDescription,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_date,COS_SIM
0,2688,30346986,LANGUR,Padded seat cover for highchair,Chairs,20953,"Breakdown of delivery vehicles en route, causi...",2024-04-09,"[{""Instruction_step"": 1, ""Instruction"": ""Deplo...",2024-05-26,0.718626
1,2689,30346986,LANGUR,Padded seat cover for highchair,Chairs,20953,"Breakdown of delivery vehicles en route, causi...",2024-04-09,"[{""Instruction_step"": 1, ""Instruction"": ""Deplo...",2024-05-26,0.718626
2,2688,30346986,LANGUR,Padded seat cover for highchair,Chairs,20953,"Breakdown of delivery vehicles en route, causi...",2024-04-09,"[{""Instruction_step"": 1, ""Instruction"": ""Deplo...",2024-05-26,0.718626
3,2689,30346986,LANGUR,Padded seat cover for highchair,Chairs,20953,"Breakdown of delivery vehicles en route, causi...",2024-04-09,"[{""Instruction_step"": 1, ""Instruction"": ""Deplo...",2024-05-26,0.718626
4,2595,40240989,DAGOTTO,Foot-rest,Chairs,14397,"Breakdown of delivery vehicles en route, causi...",2024-04-01,"[{""Instruction_step"": 1, ""Instruction"": ""Deplo...",2024-05-15,0.720749


In [25]:
# Test the vector search
query =  "What is the quality issue for packing defects for cabinet?"
df_context_en = run_vector_search_en(query=query, metric="L2DISTANCE",k=5)
df_context_en

,BlockID,MaterialNumber,MaterialName,MaterialDescription,PurchasingGroupDescription,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_date,L2D_SIM
0,1803,30396429,LIXHULT,"Cabinet, 35x60 cm",Cabinets & cupboards,18929,Identification of packaging defects that compr...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-05-06,0.54918
1,1804,30396429,LIXHULT,"Cabinet, 35x60 cm",Cabinets & cupboards,18929,Identification of packaging defects that compr...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-05-06,0.54918
2,1802,30396429,LIXHULT,"Cabinet, 35x60 cm",Cabinets & cupboards,18929,Identification of packaging defects that compr...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-05-06,0.54918
3,1803,30396429,LIXHULT,"Cabinet, 35x60 cm",Cabinets & cupboards,18929,Identification of packaging defects that compr...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-05-06,0.54918
4,1804,30396429,LIXHULT,"Cabinet, 35x60 cm",Cabinets & cupboards,18929,Identification of packaging defects that compr...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",2024-05-06,0.54918


In [26]:
# Test the vector search
query = "의자의 포장불량에 대한 문제는 무엇입니까?"
df_context_kr = run_vector_search_kr(query=query, metric="L2DISTANCE",k=5)
df_context_kr

,BlockID,MaterialNumber,MaterialName,MaterialDescription_KR,PurchasingGroupDescription_KR,Blocked_stock,Block_reason_KR,Blocked_date,Solution_KR,Solution_date,L2D_SIM
0,1800,30396429,LIXHULT,"캐비닛, 35x60 cm",캐비닛 및 컵보드,18929,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-05-06,0.725392
1,1800,30396429,LIXHULT,"캐비닛, 35x60 cm",캐비닛 및 컵보드,18929,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-05-06,0.725392
2,1803,30396429,LIXHULT,"캐비닛, 35x60 cm",캐비닛 및 컵보드,18929,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-05-06,0.725479
3,1804,30396429,LIXHULT,"캐비닛, 35x60 cm",캐비닛 및 컵보드,18929,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별함으로써 즉각적인 수정 조치가 ...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-05-06,0.725479
4,1802,30396429,LIXHULT,"캐비닛, 35x60 cm",캐비닛 및 컵보드,18929,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,2024-04-13,"[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-05-06,0.725479


In [27]:
# Test the vector search
query = "의자의 재고 불일치"
df_context_kr = run_vector_search_kr(query=query, metric="L2DISTANCE",k=5)
df_context_kr

,BlockID,MaterialNumber,MaterialName,MaterialDescription_KR,PurchasingGroupDescription_KR,Blocked_stock,Block_reason_KR,Blocked_date,Solution_KR,Solution_date,L2D_SIM
0,2283,30320726,ID√ÖSEN,"문과 서랍이 있는 캐비닛, 80x47x119 cm",캐비닛 및 컵보드,2441,재고 감사 중에 재고 불일치가 감지되었습니다.,2024-03-27,"[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-07-24,0.691677
1,2283,30320726,ID√ÖSEN,"문과 서랍이 있는 캐비닛, 80x47x119 cm",캐비닛 및 컵보드,2441,재고 감사 중에 재고 불일치가 감지되었습니다.,2024-03-27,"[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-07-24,0.691677
2,7354,60294797,HANVIKEN,"서랍 앞면, 60x26 cm",TV 및 미디어 가구,10570,재고 감사 중에 재고 불일치가 감지되었습니다.,2024-02-19,"[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-03-15,0.695801
3,7354,60294797,HANVIKEN,"서랍 앞면, 60x26 cm",TV 및 미디어 가구,10570,재고 감사 중에 재고 불일치가 감지되었습니다.,2024-02-19,"[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-03-15,0.695801
4,2030,50443627,NOTVIKEN,"서랍 앞면, 60x26 cm",캐비닛 및 컵보드,7710,"정기적인 재고 감사 중에 재고 불일치를 탐지하여, 개선된 재고 관리 방법의 필요성을...",2024-03-31,"[{""Instruction_step"": 1, ""Instruction_KR"": ""재고...",2024-06-13,0.698912


In [28]:
# Test the vector search
query = "의자의 재고 불일치"
df_context_kr = run_vector_search_kr(query=query, metric="COSINE_SIMILARITY",k=5)
df_context_kr

,BlockID,MaterialNumber,MaterialName,MaterialDescription_KR,PurchasingGroupDescription_KR,Blocked_stock,Block_reason_KR,Blocked_date,Solution_KR,Solution_date,COS_SIM
0,7346,347888,EDVALLA,"손잡이 떨어뜨리기, 18mm",TV 및 미디어 가구,11585,추가 문서 작성이 필요한 일부 화물로 인해 통관 보류가 연장되어 배송 일정에 영향을...,2024-06-06,"[{""Instruction_step"": 1, ""Instruction_KR"": ""관세...",2024-07-10,0.678946
1,7347,347888,EDVALLA,"손잡이 떨어뜨리기, 18mm",TV 및 미디어 가구,11585,추가 서류가 필요한 일부 화물로 인해 관세 보류가 연장되어 배송 일정에 영향을 미칩니다.,2024-06-06,"[{""Instruction_step"": 1, ""Instruction_KR"": ""관세...",2024-07-10,0.678946
2,7346,347888,EDVALLA,"손잡이 떨어뜨리기, 18mm",TV 및 미디어 가구,11585,추가 문서 작성이 필요한 일부 화물로 인해 통관 보류가 연장되어 배송 일정에 영향을...,2024-06-06,"[{""Instruction_step"": 1, ""Instruction_KR"": ""관세...",2024-07-10,0.678946
3,7347,347888,EDVALLA,"손잡이 떨어뜨리기, 18mm",TV 및 미디어 가구,11585,추가 서류가 필요한 일부 화물로 인해 관세 보류가 연장되어 배송 일정에 영향을 미칩니다.,2024-06-06,"[{""Instruction_step"": 1, ""Instruction_KR"": ""관세...",2024-07-10,0.678946
4,3789,50298418,FLISAT,"어린이용 테이블, 83x58 cm",아동용 가구,2962,포장 품질에 대한 중요한 피드백이 포장 재료와 과정의 검토로 이어집니다.,2024-04-15,"[{""Instruction_step"": 1, ""Instruction_KR"": ""중요...",2024-07-04,0.679454


### STEP 5: LangChain - Contextualize response from LLM with data in HANA Cloud

#### 5.1 Create Prompt Template

In [ ]:
'''This is output JSON format: 
{
  "SolutionDescription": [
    {
      "Step": "Step1",
      "Instruction": "Inspect the current packaging process for the electronics materials to identify the source of the defects.",
      "ResponsibleDepartment": "Quality Control"
    },
    {
      "Step": "Step2",
      "Instruction": "Implement a quality control system to ensure that all materials are properly packaged before delivery.",
      "ResponsibleDepartment": "Quality Control"
    },
    {
      "Step": "Step3",
      "Instruction": "Repackage any electronics materials that were improperly packaged.",
      "ResponsibleDepartment": "Warehouse Management"
    },
    {
      "Step": "Step4",
      "Instruction": "Train warehouse staff on proper packaging techniques to prevent future defects.",
      "ResponsibleDepartment": "Warehouse Management"
    },
    {
      "Step": "Step5",
      "Instruction": "Monitor the packaging process regularly to ensure that the solution is effective.",
      "ResponsibleDepartment": "Quality Control"
    }
  ]
}'''

In [1]:
#Answer format should follows below templates and answers refer context: 
promptTemplate_fstring = """
You are provided multiple context items that are related to the prompt you have to answer.
Use the following pieces of context to answer the question at the end. 
Context:
{context}

Question:
{query}

Answer: 
To answer questions, Please provides solutions with detail instructions with steps and ResponsibleDepartment. 
You must format your output as a plain text that adheres in english. 
This is example formate: 
1. Solution: 
- Responsible department: 
- Instructions:
1) , 2)  , 3), ...
2. Solution: 
- Responsible department: 
- Instructions:
1) , 2)  , 3), ...
...
"""
promptTemplate_fstring_KR = """
You are provided multiple context items that are related to the prompt you have to answer.
Use the following pieces of context to answer the question at the end. 
Context:
{context}

Question:
{query}

Answer: 
To answer questions, Please provides Solution_KR with detail instructions with steps and ResponsibleDepartment_KR for each step. 
You must format your output as a plain text adheres in korean. 
This is example format: 
1. 해결방안: 
- 담당부서: 
- 해결철자:
1) , 2)  , 3), ...
2. 해결방안: 
- 담당부서: 
- 해결철자:
1) , 2)  , 3), ...
...
"""


#### 5.2 Create RAG Chatbot with LLM

In [37]:

from langchain import PromptTemplate
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI

from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
proxy_client = get_proxy_client('gen-ai-hub')

promptTemplate = PromptTemplate.from_template(promptTemplate_fstring)

def ask_llm_en(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 5) -> str:
    class color:
        RED = '\033[91m'
        BLUE = '\033[94m'
        BOLD = '\033[1m'
        END = '\033[0m'
    context = ''
    if retrieval_augmented_generation == True:
        df_context = run_vector_search_en(query, metric, k)
        
        # print(color.RED + '\nHANA vector search returned {k} best matching documents.'.format(k=k) + color.END)
        # print(color.RED + '\nGenerating LLM prompt using the context information.' + color.END)
        print(df_context)
    else:
        print(color.RED + 'Generating LLM prompt WITHOUT context information.' + color.END)
        
    prompt = promptTemplate.format(query=query, context=' '.join(df_context[["BlockID","MaterialNumber", "MaterialName","MaterialDescription","PurchasingGroupDescription",
           "Block_reason","Blocked_date", "Solution", "Solution_date"]].astype('string')))
    
    # print(color.RED + '\nAsking LLM...' + color.END)
    
    llm = ChatOpenAI(deployment_id="d03974e89ef130ad", temperature=0.2) 
    response = llm.predict(prompt)
    
    print("PROPOSAL SOLUTIONS")
    print(response)

# Ask the user for their name
# query = input("Enter your questions: ")  
# print(f"Your questions is: {query}!")

query =  "What can be solution for the delivery for packing defects for furniture?"
response = ask_llm_en(query=query, retrieval_augmented_generation=True, k=5) #COSINE_SIMILARITY, L2DISTANCE

print(response)

   BlockID  MaterialNumber MaterialName  \
0      137        29179554       RAMSTA   
1      138        29179554       RAMSTA   
2      137        29179554       RAMSTA   
3      138        29179554       RAMSTA   
4     1344        70370352     MOSSARYD   

                               MaterialDescription  \
0  Day-bed frame with slatted bed base,  90x200 cm   
1  Day-bed frame with slatted bed base,  90x200 cm   
2  Day-bed frame with slatted bed base,  90x200 cm   
3  Day-bed frame with slatted bed base,  90x200 cm   
4                              Drop handle,  49 mm   

   PurchasingGroupDescription  Blocked_stock  \
0                        Beds           5872   
1                        Beds           5872   
2                        Beds           5872   
3                        Beds           5872   
4  Bookcases & shelving units           7743   

                                        Block_reason Blocked_date  \
0         Customs hold for additional documentation.   202

In [32]:
#Answer format should follows below templates and each sale price refer context: 
from langchain.prompts import PromptTemplate
promptTemplate = PromptTemplate.from_template(promptTemplate_fstring_kr)

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI

def ask_llm_kr(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 5) -> str:
    class color:
        RED = '\033[91m'
        BLUE = '\033[94m'
        BOLD = '\033[1m'
        END = '\033[0m'
    context = ''
    if retrieval_augmented_generation == True:
        # print(color.RED + 'Running retrieval augmented generation.' + color.END)
        # print(color.RED + '\nEmbedding the query string and running HANA vector search.' + color.END)
        df_context = run_vector_search_kr(query, metric, k)
        print(df_context)
        # print(color.RED + '\nHANA vector search returned {k} best matching documents.'.format(k=k) + color.END)
        # print(color.RED + '\nGenerating LLM prompt using the context information.' + color.END)
    # else:
        # print(color.RED + 'Generating LLM prompt WITHOUT context information.' + color.END)
    prompt = promptTemplate.format(query=query, context=' '.join(df_context[["BlockID","MaterialNumber", "MaterialName","MaterialDescription_KR","PurchasingGroupDescription_KR",
            "Block_reason_KR","Blocked_date", "Solution_KR", "Solution_date"]].astype('string')))
    # print(color.RED + '\nAsking LLM...' + color.END)
    
    llm = ChatOpenAI(deployment_id="d31f88c3e2e7eda5", temperature=0) 
    response = llm.predict(prompt)
    # print(color.RED + '...completed.' + color.END)
    # print(color.RED + '\nQuery: ' + color.END, query)
    # print(color.BLUE + '\nResponse:' + color.BLUE)
    print(response)


In [33]:

# Ask the user for their name
# query = input("Enter your questions: ")
# print(f"Your questions is: {query}!")

query =  "가구 포장 불량 배송에 대한 해결방안은 무엇인가요?"
response = ask_llm_kr(query=query, retrieval_augmented_generation=True, k=5) #COSINE_SIMILARITY, L2DISTANCE
print(response)

   BlockID  MaterialNumber MaterialName MaterialDescription_KR  \
0     3024        90246291     INGATORP             팔걸이가 있는 의자   
1     3023        90246291     INGATORP             팔걸이가 있는 의자   
2     3025        90246291     INGATORP             팔걸이가 있는 의자   
3     3024        90246291     INGATORP             팔걸이가 있는 의자   
4     3023        90246291     INGATORP             팔걸이가 있는 의자   

  PurchasingGroupDescription_KR  Blocked_stock  \
0                           의자들          27528   
1                           의자들          27528   
2                           의자들          27528   
3                           의자들          27528   
4                           의자들          27528   

                                   Block_reason_KR Blocked_date  \
0  안전 문제로 특정 제품 배치에 대한 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.   2024-04-18   
1     안전 문제로 특정 제품 배치의 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.   2024-04-18   
2     안전 문제로 특정 제품 배치의 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.   2024-04-18   
3  안전 문제로 특정 제품 배치에 대한 긴급 리콜이 발생하여

### STEP 6: Ask Questions & Get Answer

In [34]:
query =  "가구 포장 불량 배송에 대한 해결방안은 무엇인가요?"
response = ask_llm_kr(query=query, retrieval_augmented_generation=True, k=4) #COSINE_SIMILARITY, L2DISTANCE
# response = ask_llm_en(query=query, retrieval_augmented_generation=True, k=4)
print(response)

   BlockID  MaterialNumber MaterialName MaterialDescription_KR  \
0     3028        90246291     INGATORP             팔걸이가 있는 의자   
1     3028        90246291     INGATORP             팔걸이가 있는 의자   
2     3029        90246291     INGATORP             팔걸이가 있는 의자   
3     3029        90246291     INGATORP             팔걸이가 있는 의자   

  PurchasingGroupDescription_KR  Blocked_stock  \
0                           의자들          27528   
1                           의자들          27528   
2                           의자들          27528   
3                           의자들          27528   

                                   Block_reason_KR Blocked_date  \
0  안전 문제로 특정 제품 배치에 대한 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.   2024-04-18   
1  안전 문제로 특정 제품 배치에 대한 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.   2024-04-18   
2     안전 문제로 특정 제품 배치의 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.   2024-04-18   
3     안전 문제로 특정 제품 배치의 긴급 리콜이 발생하여 소비자의 안전을 보장합니다.   2024-04-18   

                                         Solution_KR Solution_date   COS_SIM  
0  

In [38]:
query =  "What can be solution for the delivery for packing defects for furniture"
response = ask_llm_en(query=query, retrieval_augmented_generation=True, k=4) 
print(response)

   BlockID  MaterialNumber MaterialName  \
0     4692        69017475       KALLAX   
1     4692        69017475       KALLAX   
2     4693        69017475       KALLAX   
3     4693        69017475       KALLAX   

                         MaterialDescription PurchasingGroupDescription  \
0  Shelving unit with 8 inserts,  147x147 cm              Room dividers   
1  Shelving unit with 8 inserts,  147x147 cm              Room dividers   
2  Shelving unit with 8 inserts,  147x147 cm              Room dividers   
3  Shelving unit with 8 inserts,  147x147 cm              Room dividers   

   Blocked_stock                                       Block_reason  \
0          17764  Safety concerns trigger an urgent recall of sp...   
1          17764  Safety concerns trigger an urgent recall of sp...   
2          17764  Safety concerns trigger an urgent recall of sp...   
3          17764  Safety concerns trigger an urgent recall of sp...   

  Blocked_date                                      